In [1]:
#Data Collection
import nltk
from nltk.corpus import gutenberg
import pandas as pd
nltk.download('gutenberg')

# load the dataset
data=gutenberg.raw('shakespeare-hamlet.txt')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [2]:
#save file
with open('hemlet.txt','w') as file:
    file.write(data)


In [3]:
#Data preprocessing
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

#load the dataset
with open('hemlet.txt','r') as file:
    text = file.read().lower()

#tokenize the word(creating indexes for each words)
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
#count no of words
total_words=len(tokenizer.word_index)+1 #total_words===vocab size
total_words

4818

In [4]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'i': 5,
 'you': 6,
 'a': 7,
 'my': 8,
 'it': 9,
 'in': 10,
 'that': 11,
 'ham': 12,
 'is': 13,
 'not': 14,
 'his': 15,
 'this': 16,
 'with': 17,
 'your': 18,
 'but': 19,
 'for': 20,
 'me': 21,
 'lord': 22,
 'as': 23,
 'what': 24,
 'he': 25,
 'be': 26,
 'so': 27,
 'him': 28,
 'haue': 29,
 'king': 30,
 'will': 31,
 'no': 32,
 'our': 33,
 'we': 34,
 'on': 35,
 'are': 36,
 'if': 37,
 'all': 38,
 'then': 39,
 'shall': 40,
 'by': 41,
 'thou': 42,
 'come': 43,
 'or': 44,
 'hamlet': 45,
 'good': 46,
 'do': 47,
 'hor': 48,
 'her': 49,
 'let': 50,
 'now': 51,
 'thy': 52,
 'how': 53,
 'more': 54,
 'they': 55,
 'from': 56,
 'enter': 57,
 'at': 58,
 'was': 59,
 'oh': 60,
 'like': 61,
 'most': 62,
 'there': 63,
 'well': 64,
 'know': 65,
 'selfe': 66,
 'would': 67,
 'them': 68,
 'loue': 69,
 'may': 70,
 "'tis": 71,
 'vs': 72,
 'sir': 73,
 'qu': 74,
 'which': 75,
 'did': 76,
 'why': 77,
 'laer': 78,
 'giue': 79,
 'thee': 80,
 'ile': 81,
 'must': 82,
 'hath': 

In [6]:
inuputsequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        inuputsequences.append(n_gram_sequence)

In [7]:
#max length of words in sentence to create pad_sequences
max_length=max([len(x) for x in inuputsequences])
max_length


14

In [8]:
input_sequences=np.array(pad_sequences(inuputsequences,maxlen=max_length,padding='pre'))

In [9]:
#divide into train test set
X,y = input_sequences[:,:-1],input_sequences[:,-1]

In [10]:
# Print shapes for debugging
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("total_words:", total_words)
print("max_length:", max_length)

Shape of X: (25732, 13)
Shape of y: (25732,)
total_words: 4818
max_length: 14


In [11]:
#convert y into categorical
import tensorflow as tf
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
x_train,x_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [13]:
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='.*deprecated.*')
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.regularizers import l2

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Assuming total_words and max_length are already defined
# Define the model
model = Sequential([
    Embedding(total_words, 100, input_length=max_length),
    LSTM(150, return_sequences=True),
    Dropout(0.2),
    LSTM(100),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Build the model
model.build(input_shape=(None, max_length))

# Now, the model will be properly built and the summary will display the parameter counts.
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 14, 100)             │         481,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 14, 150)             │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │         100,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4818)                │         486,618 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [15]:

# Assuming you have your data prepared as X_train, y_train, X_val, y_val
history = model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=128,
    validation_data=(x_test, y_test),
)

Epoch 1/100
161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 152ms/step - accuracy: 0.0271 - loss: 7.7503 - val_accuracy: 0.0336 - val_loss: 7.4063
Epoch 2/100
161/161 ━━━━━━━━━━━━━━━━━━━━ 39s 143ms/step - accuracy: 0.0257 - loss: 7.0451 - val_accuracy: 0.0251 - val_loss: 7.5280
Epoch 3/100
161/161 ━━━━━━━━━━━━━━━━━━━━ 41s 145ms/step - accuracy: 0.0274 - loss: 7.0601 - val_accuracy: 0.0336 - val_loss: 7.5983
Epoch 4/100
161/161 ━━━━━━━━━━━━━━━━━━━━ 21s 133ms/step - accuracy: 0.0236 - loss: 7.0454 - val_accuracy: 0.0336 - val_loss: 7.5688
Epoch 5/100
161/161 ━━━━━━━━━━━━━━━━━━━━ 24s 149ms/step - accuracy: 0.0268 - loss: 7.0402 - val_accuracy: 0.0336 - val_loss: 7.5108
Epoch 6/100
161/161 ━━━━━━━━━━━━━━━━━━━━ 40s 144ms/step - accuracy: 0.0306 - loss: 6.9614 - val_accuracy: 0.0336 - val_loss: 7.4207
Epoch 7/100
161/161 ━━━━━━━━━━━━━━━━━━━━ 21s 133ms/step - accuracy: 0.0299 - loss: 6.9066 - val_accuracy: 0.0336 - val_loss: 7.2085
Epoch 8/100
161/161 ━━━━━━━━━━━━━━━━━━━━ 43s 144ms/step - accuracy: 0.0314 -

In [19]:
import tensorflow as tf
from tensorflow.keras.models import save_model, load_model
import warnings

# Assuming 'model' is your trained model

# Temporarily suppress the warning
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')

# Save the model in H5 format
model.save('hamlet_lstm_model.h5')

# Restore warnings
warnings.filterwarnings('default', category=UserWarning, module='tensorflow')

print("Model saved successfully in .h5 format.")

Model saved successfully in .h5 format.


In [20]:
# To download the file in Colab, run this in a cell:
from google.colab import files
files.download('hamlet_lstm_model.h5')

# Don't forget to save your tokenizer as well
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
files.download('tokenizer.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# To download the file, run this in a cell:
from google.colab import files
files.download('hamlet_lstm_model.h5')

In [25]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted, axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [22]:
tensorflow.__version__

'2.17.0'

In [27]:
# Example usage:
seed_text = "To be or not to be"
next_words = 5
generated_text = generate_text(seed_text, next_words, model, max_length)
print(generated_text)

# To predict just the next word:
next_word = generate_text(seed_text, 1, model, max_length).split()[-1]
print(f"Next word: {next_word}")

To be or not to be that adulterate opposed may beware
Next word: that
